환경설정


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

참고문서: https://python.langchain.com/docs/modules/data_connection/document_transformers/


## 텍스트 분할기

문서를 로드한 후에는 애플리케이션에 더 적합하도록 문서를 변형하고 싶을 때가 많습니다. 가장 간단한 예로, 긴 문서를 모델의 컨텍스트 창에 맞도록 작은 덩어리로 분할하고 싶을 수 있습니다.

LangChain에는 문서를 쉽게 분할, 결합, 필터링 및 기타 조작할 수 있는 여러 가지 문서 변환기가 내장되어 있습니다.

긴 텍스트를 다루고 싶을 때는 해당 텍스트를 여러 조각으로 분할해야 합니다. 간단하게 들리지만 여기에는 많은 잠재적 복잡성이 있습니다. 이상적으로는 의미적으로 관련된 텍스트 조각을 함께 보관하는 것이 좋습니다. "의미적으로 연관된"이란 텍스트 유형에 따라 달라질 수 있습니다. 이 노트북에서는 이를 위한 몇 가지 방법을 보여드립니다.

큰 틀에서 텍스트 분할기는 다음과 같이 작동합니다.

- 텍스트를 의미적으로 의미 있는 작은 덩어리(주로 문장)로 나눕니다.
- 특정 크기(특정 함수로 측정한 크기)에 도달할 때까지 이 작은 청크들을 더 큰 청크로 결합하기 시작합니다.
- 그 크기에 도달하면 그 청크를 자체 텍스트 조각으로 만든 다음, 청크 사이의 맥락을 유지하기 위해 약간의 겹침이 있는 새로운 텍스트 청크를 만들기 시작합니다.

즉, 텍스트 분할기를 사용자 지정할 수 있는 두 개의 다른 축이 있다는 뜻입니다.

- 텍스트 분할 방식
- 청크 크기를 측정하는 방법


## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`


## PDF 문서 로드를 위한 다양한 방법


## PyPDFLoader


가장 일반적으로 많이 활용되는 방법이며, 대부분의 일반적인 PDF 파일을 문제 없이 불러 올 수 있습니다.


### ① loader.load()

문서를 페이지 단위로 불러옵니다. `document` 변수에는 페이지 별 `Document` 객체가 리스트 형태로 존재하며, **1개 Document == PDF 문서의 1개 페이지** 를 의미합니다.


In [3]:
# 파일경로
filepath = "data/SPRI_AI_Brief_2023년12월호_F.pdf"

In [4]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader(filepath)

# 페이지 별 문서 로드
document = loader.load()

print(f"문서의 수: {len(document)}")

문서의 수: 23


로드한 `document` 의 내용을 확인해 보면 다음과 같습니다.


In [5]:
# page_content 에는 본문의 내용이 있음
print(document[0].page_content[:200])  # 일부내용 출력
print("===" * 10)
# metadata 출력
print(document[0].metadata)

2023 년 12월호
{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 0}


In [6]:
# PDF문서의 페이지 수 == len(document)
print(f"문서의 수: {len(document)}")

문서의 수: 23


### ② TextSplitter 활용


문서를 로드시 페이지 단위로 분리 저장하는 방법이 아닌, chunk_size 단위로 저장할 수 있는데, chunk_size 단위로 분리/저장하기 위해서는 다음과 같이 `TextSplitter` 를 활용할 수 있습니다.

- `chunk_size`: 하나의 chunk(단위) 당 보관할 토큰 수
- `chunk_overlap`: chunk 간 겹쳐지는 토큰의 개수


In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader(filepath)
print(loader)

### CharacterTextSplitter

- [API 문서](https://api.python.langchain.com/en/stable/text_splitter/langchain.text_splitter.CharacterTextSplitter.html?highlight=charactertext#langchain.text_splitter.CharacterTextSplitter)

가장 간단한 방법입니다. 이 방법은 문자(기본값은 "")를 기준으로 분할하고 문자 수에 따라 Chunk 의 길이를 측정합니다.

- 텍스트 분할 방법: Character
- Chunk 크기 측정 방법: Character


In [17]:
text = "안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다."

In [18]:
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator=".")
text_splitter.split_text(text)

['안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다']

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=10, separator=" ")
text_splitter.split_text(text)

['안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다.',
 '이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은',
 '제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제',
 '반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요.',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.']

In [11]:
# splitter 정의
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# 문서 로드 및 분할 (load_and_split)
split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"CharacterTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

CharacterTextSplitter 		 사용시 문서의 수: 23


In [12]:
split_docs[3].page_content

'1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 ’에 서명하고 \n광범위한 행정 조치를 명시\nn행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함KEY Contents\n£바이든 대통령 , AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄\nn(AI 안전과 보안 기준) 강력한 AI 시스템을 개발하는 기업에게 안전 테스트 결과와 시스템에 관한 \n주요 정보를 미국 정부와 공유할 것을 요구하고 , AI 시스템의 안전성과 신뢰성 확인을 위한 표준 및 \nAI 생성 콘텐츠 표시를 위한 표준과 모범사례 확립을 추진\n∙△1026 플롭스 (FLOPS, Floating Point Operation Per Second) 를 초과하는 컴퓨팅 성능 또는 생물학적 \n서열 데이터를 주로 사용하고 1023플롭스를 초과하는 컴퓨팅 성능을 사용하는 모델 △단일 데이터센터에서 \n1,000Gbit/s 이상의 네트워킹으로 연결되며 AI 훈련에서 이론상 최대 1020 플롭스를 처리할 수 있는 \n컴퓨팅 용량을 갖춘 컴퓨팅 클러스터가 정보공유 요구대상\nn(형평성과 시민권 향상) 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견 및 기타 \n문제를 방지하는 조치를 확대\n∙형사사법 시스템에서 AI 사용 모범사례를 개발하고 , 주택 임대 시 AI 알고리즘 차별을 막

In [13]:
# splitter 정의
text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50, separator="\n")

# 문서 로드 및 분할 (load_and_split)
split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"CharacterTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

CharacterTextSplitter 		 사용시 문서의 수: 43


In [44]:
split_docs[5].page_content

'1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 ’에 서명하고 \n광범위한 행정 조치를 명시\nn행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함KEY Contents\n£바이든 대통령 , AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄\nn(AI 안전과 보안 기준) 강력한 AI 시스템을 개발하는 기업에게 안전 테스트 결과와 시스템에 관한 \n주요 정보를 미국 정부와 공유할 것을 요구하고 , AI 시스템의 안전성과 신뢰성 확인을 위한 표준 및 \nAI 생성 콘텐츠 표시를 위한 표준과 모범사례 확립을 추진\n∙△1026 플롭스 (FLOPS, Floating Point Operation Per Second) 를 초과하는 컴퓨팅 성능 또는 생물학적 \n서열 데이터를 주로 사용하고 1023플롭스를 초과하는 컴퓨팅 성능을 사용하는 모델 △단일 데이터센터에서 \n1,000Gbit/s 이상의 네트워킹으로 연결되며 AI 훈련에서 이론상 최대 1020 플롭스를 처리할 수 있는 \n컴퓨팅 용량을 갖춘 컴퓨팅 클러스터가 정보공유 요구대상\nn(형평성과 시민권 향상) 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견 및 기타 \n문제를 방지하는 조치를 확대\n∙형사사법 시스템에서 AI 사용 모범사례를 개발하고 , 주택 임대 시 AI 알고리즘 차별을 막

### RecursiveCharacterTextSplitter

- [참고문서](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter)
- [API 문서](https://api.python.langchain.com/en/stable/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html?highlight=charactertext)

이 텍스트 분할기는 일반 텍스트에 권장되는 텍스트 분할기입니다. 문자 목록으로 매개변수화됩니다. 청크가 충분히 작아질 때까지 순서대로 분할을 시도합니다. 기본 목록은 `["\n\n", "\n", " ", ""]` 입니다.

이렇게 하면 일반적으로 의미적으로 가장 연관성이 강한 텍스트 조각으로 보이는 모든 단락(그리고 문장, 단어)을 가능한 한 길게 유지하려는 효과가 있습니다.

- 텍스트를 분할하는 방법: **list of characters**
- Chunk 크기 측정 방법: **number of characters**


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)

# 문서 로드 및 분할 (load_and_split)
split_doc = loader.load_and_split(text_splitter=text_splitter)
print(f"RecursiveCharacterTextSplitter \t 사용시 문서의 수: {len(split_doc)}")

RecursiveCharacterTextSplitter 	 사용시 문서의 수: 43


In [53]:
# text = "안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요."
text = """
오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG)'에 대한 것입니다.

Retrieval Augmented Generation 

기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다. 
이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다. 
이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다. 

RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. 
이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다.
"""

text_splitter1 = CharacterTextSplitter(
    chunk_size=50, chunk_overlap=10, separator=" ")
text_splitter1.split_text(text)

["오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented",
 "Augmented Generation (RAG)'에 대한 것입니다.\n\nRetrieval",
 'Augmented Generation \n\n기존의 언어 모델에 검색 기능을 통합하여 성능을',
 '통합하여 성능을 향상시키는 방법론입니다. \n이 접근 방식에서, 모델은 먼저 관련 정보를',
 '먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다. \n이를 통해 모델은 더',
 '통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게',
 '제공할 수 있게 됩니다. \n\nRAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한',
 '특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. \n이 기술은 자연어 처리 분야에서',
 '처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등',
 '답변 시스템 등 다양한 응용 분야에 활용되고 있습니다.']

In [52]:
text_splitter2 = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)
text_splitter2.split_text(text)

["오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented",
 "Augmented Generation (RAG)'에 대한 것입니다.",
 'Retrieval Augmented Generation',
 '기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다.',
 '이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을',
 '바탕으로 응답을 생성합니다.',
 '이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수',
 '답변을 제공할 수 있게 됩니다.',
 'RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데',
 '제공하는 데 유용합니다.',
 '이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약,',
 '문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다.']

### TokenTextSplitter


In [25]:
from langchain.text_splitter import TokenTextSplitter

# splitter 정의
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"TokenTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

TokenTextSplitter 		 사용시 문서의 수: 60


In [26]:
text_splitter3 = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
)
text_splitter3.split_text(text)

['안녕하세요. 반갑습니다. 제 이름은 테디입',
 ' 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝�',
 '��로젝트입니다.안녕하세요. 반갑습니다. �',
 '습니다. 제 이름은 테디입니다. 랭체인은 정말',
 '인은 정말 좋은 프로젝트입니다.안녕하세요',
 '��하세요.']

In [ ]:
from langchain.text_splitter import KonlpyTextSplitter

text_splitter4 = KonlpyTextSplitter()
text_splitter4.split_text(text)

In [ ]:
print(split_docs[10].page_content)

## Semantic Chunker


In [54]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

In [55]:
text_splitter4 = SemanticChunker(
    OpenAIEmbeddings()
)

In [61]:
text_splitter4.split_text(text)

["\n오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG)'에 대한 것입니다. Retrieval Augmented Generation \n\n기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다. 이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다.",
 '이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다. RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. 이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다. ']

In [60]:
for sent in text_splitter4.split_text(text):
    print(sent)


오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG)'에 대한 것입니다. Retrieval Augmented Generation 

기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다. 이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다.
이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다. RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. 이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다. 
